In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm


### hyper_parameter

In [10]:
batch_size = 32
test_batch_size = 1000
iters = 10000
lr = 0.01
momentum = 0.9
alpha = 1.0
xi = 10.0
eps = 1.0
ip = 1
workers = 8 
seed = 1
log_interval = 100

### vat.py

In [11]:
import contextlib
import torch
import torch.nn as nn
import torch.nn.functional as F


@contextlib.contextmanager
def _disable_tracking_bn_stats(model):

    def switch_attr(m):
        if hasattr(m, 'track_running_stats'):
            m.track_running_stats ^= True
            
    model.apply(switch_attr)
    yield
    model.apply(switch_attr)


def _l2_normalize(d):
    d_reshaped = d.view(d.shape[0], -1, *(1 for _ in range(d.dim() - 2)))
    d /= torch.norm(d_reshaped, dim=1, keepdim=True) + 1e-8
    return d


class VATLoss(nn.Module):

    def __init__(self, xi=10.0, eps=1.0, ip=1):
        """VAT loss
        :param xi: hyperparameter of VAT (default: 10.0)
        :param eps: hyperparameter of VAT (default: 1.0)
        :param ip: iteration times of computing adv noise (default: 1)
        """
        super(VATLoss, self).__init__()
        self.xi = xi
        self.eps = eps
        self.ip = ip

    def forward(self, model, x):
        with torch.no_grad():
            pred = F.softmax(model(x), dim=1)

        # prepare random unit tensor
        d = torch.rand(x.shape).sub(0.5).to(x.device)
        d = _l2_normalize(d)

        with _disable_tracking_bn_stats(model):
            # calc adversarial direction
            for _ in range(self.ip):
                d.requires_grad_()
                pred_hat = model(x + self.xi * d)
                logp_hat = F.log_softmax(pred_hat, dim=1)
                adv_distance = F.kl_div(logp_hat, pred, reduction='batchmean')
                adv_distance.backward()
                d = _l2_normalize(d.grad)
                model.zero_grad()
    
            # calc LDS
            r_adv = d * self.eps
            pred_hat = model(x + r_adv)
            logp_hat = F.log_softmax(pred_hat, dim=1)
            lds = F.kl_div(logp_hat, pred, reduction='batchmean')

        return lds

### data_utils.py 

In [67]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms


class SimpleDataset(Dataset):

    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform

    def __getitem__(self, index):
        img = self.x[index]
        if self.transform is not None:
            img = self.transform(img)
        target = self.y[index]
        return img, target

    def __len__(self):
        return len(self.x)


class InfiniteSampler(sampler.Sampler):

    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __iter__(self):
        while True:
            order = np.random.permutation(self.num_samples)
            for i in range(self.num_samples):
                yield order[i]

    def __len__(self):
        return None


def get_iters(
        dataset='CIFAR10', root_path='.', data_transforms=None,
        n_labeled=4000, valid_size=1000,
        l_batch_size=32, ul_batch_size=128, test_batch_size=256,
        workers=0, pseudo_label=None):
    
    train_path = f'{root_path}/data/{dataset}/train/'
    test_path = f'{root_path}/data/{dataset}/test/'

    if dataset == 'CIFAR10':
        train_dataset = datasets.CIFAR10(train_path, download=True, train=True, transform=None)
        test_dataset = datasets.CIFAR10(test_path, download=True, train=False, transform=None)
    elif dataset == 'CIFAR100':
        train_dataset = datasets.CIFAR100(train_path, download=True, train=True, transform=None)
        test_dataset = datasets.CIFAR100(test_path, download=True, train=False, transform=None)
    else:
        raise ValueError

    if data_transforms is None:
        data_transforms = {
            'train': transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
            'eval': transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
        }

    x_train, y_train = train_dataset.data, np.array(train_dataset.targets)
    x_test, y_test = test_dataset.data, np.array(test_dataset.targets)

    randperm = np.random.permutation(len(x_train))
    labeled_idx = randperm[:n_labeled]
    validation_idx = randperm[n_labeled:n_labeled + valid_size]
    unlabeled_idx = randperm[n_labeled + valid_size:]

    x_labeled = x_train[labeled_idx]
    x_validation = x_train[validation_idx]
    x_unlabeled = x_train[unlabeled_idx]

    y_labeled = y_train[labeled_idx]
    y_validation = y_train[validation_idx]
    if pseudo_label is None:
        y_unlabeled = y_train[unlabeled_idx]
    else:
        assert isinstance(pseudo_label, np.ndarray)
        y_unlabeled = pseudo_label
    
    data_iterators = {
        'labeled': iter(DataLoader(
            SimpleDataset(x_labeled, y_labeled, data_transforms['train']),
            batch_size=l_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_labeled)),
        )),
        'unlabeled': iter(DataLoader(
            SimpleDataset(x_unlabeled, y_unlabeled, data_transforms['train']),
            batch_size=ul_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_unlabeled)),
        )),
        'make_pl': iter(DataLoader(
            SimpleDataset(x_unlabeled, y_unlabeled, data_transforms['eval']),
            batch_size=ul_batch_size, num_workers=workers, shuffle=False
        )),
        'val': iter(DataLoader(
            SimpleDataset(x_validation, y_validation, data_transforms['eval']),
            batch_size=len(x_validation), num_workers=workers, shuffle=False
        )),
        'test': iter(DataLoader(
            SimpleDataset(x_test, y_test, data_transforms['eval']),
            batch_size=test_batch_size, num_workers=workers, shuffle=False
        ))
    }

    return data_iterators


### utils.py

In [68]:
from collections import OrderedDict
import logging
from pathlib import Path
import torch


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, top_k=(1,)):
    """Computes the precision@k for the specified values of k"""
    max_k = max(top_k)
    batch_size = target.size(0)

    _, pred = output.topk(max_k, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in top_k:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))

    if len(res) == 1:
        res = res[0]

    return res

### vat.py

In [99]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3)
        self.fc1 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(-1, 128)
        x = self.fc1(x)
        return x
    
def train(model, device, data_iterators, optimizer):
    model.train()
    for i in tqdm(range(1000)):
        
        # reset
        if i % 100 == 0:
            ce_losses = AverageMeter()
            vat_losses = AverageMeter()
            prec1 = AverageMeter()
        
        x_l, y_l = next(data_iterators['labeled'])
        x_ul, _ = next(data_iterators['unlabeled'])

        x_l, y_l = x_l.to(device), y_l.to(device)
        x_ul = x_ul.to(device)

        optimizer.zero_grad()

        vat_loss = VATLoss(xi=10.0 , eps=1.0, ip=1)
        cross_entropy = nn.CrossEntropyLoss()

        lds = vat_loss(model, x_ul)
        output = model(x_l)
        classification_loss = cross_entropy(output, y_l)
        loss = classification_loss + 1.0 * lds
        loss.backward()
        optimizer.step()

        acc = accuracy(output, y_l)
        ce_losses.update(classification_loss.item(), x_l.shape[0])
        vat_losses.update(lds.item(), x_ul.shape[0])
        prec1.update(acc.item(), x_l.shape[0])

        if i % 100 == 0:
            print(f'\nIteration: {i}\t'
                  f'CrossEntropyLoss {ce_losses.val:.4f} ({ce_losses.avg:.4f})\t'
                  f'VATLoss {vat_losses.val:.4f} ({vat_losses.avg:.4f})\t'
                  f'Prec@1 {prec1.val:.3f} ({prec1.avg:.3f})')
            


### main

In [103]:
device = torch.device('mps')

data_iterators = get_iters(
        root_path='.',
        l_batch_size=32,
        ul_batch_size=32,
        test_batch_size=1000,
        workers=0
    )

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train(model, device, data_iterators, optimizer)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 5/10000 [00:00<13:21, 12.47it/s]  


Iteration: 0	CrossEntropyLoss 2.3205 (2.3205)	VATLoss 0.0000 (0.0000)	Prec@1 9.375 (9.375)


  1%|          | 106/10000 [00:03<04:18, 38.26it/s]


Iteration: 100	CrossEntropyLoss 2.2977 (2.2977)	VATLoss 0.0000 (0.0000)	Prec@1 21.875 (21.875)


...

 98%|█████████▊| 9806/10000 [04:15<00:05, 37.99it/s]


Iteration: 9800	CrossEntropyLoss 2.3071 (2.3071)	VATLoss 0.0000 (0.0000)	Prec@1 9.375 (9.375)


 99%|█████████▉| 9907/10000 [04:17<00:02, 37.45it/s]


Iteration: 9900	CrossEntropyLoss 2.3026 (2.3026)	VATLoss 0.0000 (0.0000)	Prec@1 9.375 (9.375)


100%|██████████| 10000/10000 [04:20<00:00, 38.42it/s]


In [104]:
def test(model, device, data_iterators):
    model.eval()
    correct = 0
    
    with torch.no_grad():
        for index , (x, y) in enumerate(data_iterators['test']):
            print(index , x.shape)
            with torch.no_grad():
                x, y = x.to(device), y.to(device)
                outputs = model(x)
            correct += torch.eq(outputs.max(dim=1)[1], y).detach().cpu().float().sum()
        test_acc = correct / len(data_iterators['test']._dataset) * 100

    print(f'\nTest Accuracy: {test_acc:.4f}%\n')

In [105]:
test(model, device, data_iterators)

0 torch.Size([1000, 3, 32, 32])
1 torch.Size([1000, 3, 32, 32])
2 torch.Size([1000, 3, 32, 32])
3 torch.Size([1000, 3, 32, 32])
4 torch.Size([1000, 3, 32, 32])
5 torch.Size([1000, 3, 32, 32])
6 torch.Size([1000, 3, 32, 32])
7 torch.Size([1000, 3, 32, 32])
8 torch.Size([1000, 3, 32, 32])
9 torch.Size([1000, 3, 32, 32])

Test Accuracy: 10.0000%



In [98]:
data_iterators['test']._next_index

<bound method _BaseDataLoaderIter._next_index of <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x177434490>>